# Rank documents with user query

## Second matching according to order

In [1]:
query = 'you can\'t decline'

In [2]:
import os

dir = os.getcwd()
file_list = os.listdir(dir)
file_list = [file for file in file_list if file.endswith('.txt')]
file_name = [name.split('.')[0] for name in file_list]

In [3]:
import re

script_list = []

for name in file_list:
    f = open(name, 'r')
    script = f.read()
    script = re.sub('[\n]', ' ', script)
    script_list.append(script)
    f.close()

In [4]:
print(len(script_list))

8


In [5]:
query = query.split(' ')

query_term_weight = {}
query_weight = []
a = 1 / (len(query) + 1)
for i in range(len(query)):
    query_term_weight[query[i]] = 0 + a * (i+1)
    query_weight.append(0 + a * (i+1))
print(query_term_weight)
print(query_weight)

{'you': 0.25, "can't": 0.5, 'decline': 0.75}
[0.25, 0.5, 0.75]


## give the same weight if the term is same

In [6]:
script_weight = []
script = []

for i in range(len(script_list)):
    script.append(script_list[i].split(' '))
print(len(script))

8


### cosine similarity

In [7]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return round(dot(A, B)/(norm(A)*norm(B)), 3)

In [8]:
a = np.asarray([0.25,0.5,0.75]) # I love you 
b = np.asarray([0.5, 0.25 ,0.75]) # love I you
c = np.asarray([0.75, 0.5, 0.25]) # you love I
d = np.asarray([0.5,0.75,0.25]) # you I love
e = np.asarray([0.25,0.75,0.5]) # I you love
f = np.asarray([0.75, 0.25, 0.5]) # love you I
print(cos_sim(a,b), cos_sim(a,c), cos_sim(a,d), cos_sim(a,e), cos_sim(a,f))

0.929 0.714 0.786 0.929 0.786


In [9]:
for i in range(len(script)):
    script_term_weight = {}
    temp = []
    for j in range(len(script[i])):
        a = 1 / (len(script[i]) + 1)
        if script[i][j] not in script_term_weight.keys():
            script_term_weight[script[i][j]] = 0 + a * (j+1)
    
    for k in range(len(query)):
        try:
            temp.append(script_term_weight[query[k]])
        except KeyError:
            temp.append(0)
    script_weight.append(temp)
print(len(script_weight))
print(script_weight)

8
[[0.2559366754617414, 0.891820580474934, 0], [0.26987951807228916, 0, 0], [0.17624521072796934, 0, 0], [0, 0, 0], [0, 0, 0], [0.8086124401913876, 0.15789473684210525, 0.16267942583732056], [0.16078431372549018, 0, 0.16862745098039217], [0.5963855421686747, 0.8373493975903615, 0.8734939759036146]]


In [10]:
similarity = []
for i in range(len(script_weight)):
    similarity.append(cos_sim(query_weight, script_weight[i]))
print(similarity)

[0.588, 0.267, 0.267, nan, nan, 0.513, 0.765, 0.969]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


## after normalize

In [20]:
normalize_weight = []
normalize_query_weight = []

query_mean = np.mean(query_weight)
query_std = np.std(query_weight)
for i in range(len(query_weight)):
    normalize_query_weight.append((query_weight[i] - query_mean) / query_std)
    
index = []
    
for i in range(len(script)):
    script_term_weight = {}
    temp = []
    
    for j in range(len(script[i])):
        a = 1 / (len(script[i]) + 1)
        if script[i][j] not in script_term_weight.keys():
            script_term_weight[script[i][j]] = 0 + a * (j+1)
    weight = list(script_term_weight.values())
    mean = np.mean(weight)
    std = np.std(weight)
    for j in range(len(weight)):
        temp.append((weight[j] - mean) / std)
        
    key = list(script_term_weight.keys())
    key_index = []
    for k in range(len(query)):
        try:
            key_index.append(key.index(query[k]))
        except ValueError:
            key_index.append(len(key) + 1)
    index.append(key_index)
    normalize_weight.append(temp)


print(normalize_query_weight)
print(index)
#print(normalize_weight)

[-1.224744871391589, 0.0, 1.224744871391589]
[[76, 175, 191], [78, 144, 144], [43, 128, 128], [137, 137, 137], [95, 95, 95], [120, 31, 32], [50, 132, 53], [80, 105, 111]]


In [31]:
normalize_doc_weight = []
for i in range(len(normalize_weight)):
    temp = []
    for j in range(len(index[i])):
        try: 
            temp.append(normalize_weight[i][index[i][j]])
        except IndexError:
            temp.append(0)
    normalize_doc_weight.append(temp)

In [33]:
normalize_similarity = []
for i in range(len(normalize_doc_weight)):
    print(normalize_doc_weight[i])
    normalize_similarity.append(cos_sim(normalize_query_weight, normalize_doc_weight[i]))
print(normalize_similarity)

[-0.47397747605467533, 1.7669547519828652, 0]
[-0.2060446196310617, 0, 0]
[-0.6698053803469881, 0, 0]
[0, 0, 0]
[0, 0, 0]
[1.7453017947889093, -0.859455191264322, -0.8403025663668718]
[-0.679837803105038, 0, -0.6412299122600136]
[0.5945251517415522, 1.4849695365319986, 1.6185361942505656]
[0.183, 0.707, 0.707, nan, nan, -0.863, 0.029, 0.318]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [34]:
print(normalize_similarity)
print(similarity)

[0.183, 0.707, 0.707, nan, nan, -0.863, 0.029, 0.318]
[0.588, 0.267, 0.267, nan, nan, 0.513, 0.765, 0.969]
